<a href="https://colab.research.google.com/github/Lemon-Aki/DeepLearningFromScratch1/blob/main/Chapter5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#오차역전파법 : 가중치 매개변수의 기울기를 효율적으로 계산, 수치 미분은 계산 시간이 오래걸림
#계산 그래프 : 오차역전파법을 설명하는 방법 중 하나. 계산 과정을 그래프로 그린것, 노드(node)와 에지(edge)로 표현
#에지 : 노드 사이의 직선
#계산 그래프를 구성한다 -> 그래프에서 계산을 왼쪽에서 오른쪽으로 진행한다(순전파)
#역전파 : 그래프에서 계산을 반대 방향(오른쪽에서 왼쪽)으로 전파
#국소적 계산 : 자신과 직접 관계된 작은 범위의 정보만으로 결과를 출력할 수 있음
#합성 함수 : 여러개로 구성된 함수
#연쇄법칙 : 합성 함수의 미분은 합성 함수를 각 구성하는 각 함수의 미분의 곱으로 나타낼 수 있음

In [2]:
# 코랩과 구글드라이드를 연동(인증 필요)
#Transport endpoint is not connected 에러시 코랩 재연결
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
%cd /gdrive/MyDrive/DeepLearningFromScratch1/

Mounted at /gdrive
/gdrive/MyDrive/DeepLearningFromScratch1


In [3]:
#단순한 계층 구현
#곱셈계층
class MulLayer : 
  def __init__(self):
    self.x = None
    self.y = None

  #순전파
  def forward(self, x, y):
    self.x = x
    self.y = y
    out = x * y

    return out
  
  #역전파
  def backward(self, dout):
    dx = dout * self.y
    dy = dout * self.x

    return dx, dy

apple = 100
apple_num = 2
tax = 1.1

#계층 생성
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

#역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

220.00000000000003
2.2 110.00000000000001 200


In [4]:
#덧셈 계층
class AddLayer:
  def __init__(self):
    pass

  def forward(self, x, y):
    out = x + y
    return out

  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1
    return dx, dy

apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

#계층 생성
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_price_layer = AddLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_price_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

#역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_price_layer.backward(dall_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)

print(price)
print(dapple, dapple_num, dorange, dorange_num, dtax)

715.0000000000001
2.2 110.00000000000001 3.3000000000000003 165.0 650


In [5]:
#Relu 계층 구현(0보다 크면 받은값을 그대로 전달, 0이하일 경우 0을 전달)
import numpy as np

class Relu:
  def __init__(self):
    #mask : True/False로 구성된 넘파이 배열
    self.mask = None

  def forward(self, x):
    self.mask = (x <= 0)
    print(self.mask)
    out = x.copy()
    out[self.mask] = 0

    return out

  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout

    return dx

x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

mask = (x<= 0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


In [6]:
#Sigmoid 계층 구현(전체합이 1이 되도록 하여 비율(확률)을 구할 수 있음)
class Sigmoid:
  #인스턴스 변수 out에 순전파의 출력을 보관, 역전파 계산 때 그 값을 사용
  def __init__(self):
    self.out = None

  def forward(self, x):
    out = 1 / (1 + exp(-x))
    self.out = out

    return out

  def backward(self, dout):
    dx = dout * (1.0 - self.out) * self.out

    return dx

In [7]:
#Affine 계층 구현(신경망의 순전파 때 수행하는 행렬의 곱을 기하학에서 어파인 변환이라고 함)
class Affine:
  def __init__(self, W, b):
    self.W = W
    self.b = b
    self.x = None
    self.dW = None
    self.db = None

  def forward(self, x):
    out = np.dot(x, self.W) + self.b

    return out

  def backward(self, dout):
    dx = np.dot(dout, self.W.T)
    self.dW = np.dot(self.x.T, dout)
    self.db = np.sum(dout, axis = 0)

In [8]:
#Softmax-with-Loss 계층 : 입력값을 정규화하여 출력
#분류될 클래스가 n개라 할 때, n차원의 벡터를 입력받아, 각 클래스에 속할 확률을 추정
from common.functions import cross_entropy_error
class SoftmaxWithLoss:
  def __init__(self):
    self.loss = None    #손실
    self.y = None     #softmax 출력값
    self.t = None     #정답 레이블(원-핫 벡트)

  def forward(self, x, t):
    self.t = t
    self.y = softmax(x)
    self.loss = cross_entropy_error(self.y, self.t)
    return self.loss

  def backward(self, dout=1):
    batch_size = self.t.shape[0]
    dx = (self.y - self.t) / batch_size

    return dx

In [13]:
#신경망 학습의 전체 그림
#전체 : 신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 학습이라 함
#학습 1단계 - 미니배치
#훈련 데이터 중 일부를 무작위로 가져옴. 이 미니배치의 손실 함수 값을 줄이는 것이 목표
#학습 2단계 - 기울기 산출
#미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구함. 기울기는 손실 함수의 값을 가장 작게하는 방향을 제시
#학습 3단계 - 매개변수 갱신
#가중치 매개변수를 기울기 방향으로 아주 조금 갱신
#학습 4단계 - 1~3단계 반복
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:

  def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
    
    #가중치 초기화
    self.params = {}
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
    self.params['b1'] = np.zeros(hidden_size)
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
    self.params['b2'] = np.zeros(output_size)

    #계층 생성
    #OrderedDict() : 순서가 있는 딕셔너리
    self.layers = OrderedDict()
    self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
    self.layers['Relu1'] = Relu()
    self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
    self.lastLayer = SoftmaxWithLoss()

  def predict(self, x):
    for layer in self.layers.values():
      x = layer.forward(x)

    return x

  def loss(self, x, t):
    y = self.predict(x)
    return self.lastLayer.forward(y, t)

  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis=1)
    if t.ndim != 1 : t = np.argmax(t, axis=1)

    accuracy = np.sum(y == t) / float(x.shape[0])
    return accuracy

  def numerical_gradient(self, x, t):
    loss_W = lambda W: self.loss(x, t)

    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
    return grads

  def gradient(self, x, t):
    #순전파
    self.loss(x, t)

    #역전파
    dout = 1
    dout = self.lastLayer.backward(dout)

    layers = list(self.layers.values())
    layers.reverse()
    for layer in layers:
      dout = layer.backward(dout)

    #결과 저장
    grads = {}
    grads['W1'] = self.layers['Affine1'].dW
    grads['b1'] = self.layers['Affine1'].db
    grads['W2'] = self.layers['Affine2'].dW
    grads['b2'] = self.layers['Affine2'].db

    return grads

In [15]:
#기울기 확인 : 수치미분, 오차역전파법으로 구현한 기울기가 일치하는지 확인하는 작업
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

#각 가중치의 차이의 절댓값을 구한 후, 그 절댓값들의 평균을 낸다
for key in grad_numerical.keys():
  diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
  print(key + ' : ' + str(diff))

W1 : 5.355552638790701e-10
b1 : 3.2651183181871877e-09
W2 : 7.261029676085967e-09
b2 : 1.4011373938999095e-07


In [18]:
#오차역전파법을 사용한 학습 구현
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
  batch_mask = np.random.choice(train_size, batch_size)
  x_batch = x_train[batch_mask]
  t_batch = t_train[batch_mask]

  #오차역전파법으로 기울기를 구함
  grad = network.gradient(x_batch, t_batch)

  #갱신
  for key in ('W1', 'b1', 'W2', 'b2'):
    network.params[key] -= learning_rate * grad[key]

  loss = network.loss(x_batch, t_batch)
  train_loss_list.append(loss)

  if i % iter_per_epoch ==0:
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
    print(train_acc, test_acc) 

0.10863333333333333 0.1049
0.90685 0.9092
0.9265666666666666 0.9283
0.9370833333333334 0.9354
0.9454166666666667 0.9452
0.9514833333333333 0.9494
0.9563166666666667 0.9529
0.9610333333333333 0.9572
0.9635 0.9583
0.96695 0.9601
0.9690833333333333 0.9637
0.9715333333333334 0.9658
0.9727333333333333 0.9679
0.9750833333333333 0.9662
0.9761333333333333 0.9687
0.97695 0.968
0.9782666666666666 0.9705
